In [ ]:
!pip install Pillow

In [11]:
from tensorflow.keras.utils import to_categorical  
from tensorflow.keras.models import Sequential      
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import os

In [12]:
pwd_train = 'trainDataset/splitdataset48x48/train'
pwd_val = 'trainDataset/splitdataset48x48/val'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_generator = train_datagen.flow_from_directory(
    pwd_train, 
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

validation_generator = val_datagen.flow_from_directory(
    pwd_val,  
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

In [ ]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

In [15]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(8, activation='softmax'))

In [16]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [17]:
!rm -rf Logs
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [ ]:

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  
    epochs=40,  
    validation_data=validation_generator,  
    validation_steps=validation_generator.samples // batch_size,  
    callbacks=[tensorboard_callback]  
)

In [ ]:
model_json = model.to_json()
with open("model/signlanguagedetectionmodel48x48.json",'w') as json_file:
    json_file.write(model_json)
model.save("model/signlanguagedetectionmodel48x48.h5")